In [1]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
msft.info

{'zip': '98052',
 'sector': 'Technology',
 'fullTimeEmployees': 144000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); and Skype, Outlook.com, and OneDrive. It also provides LinkedIn that includes Talent and marketing solutions, and subscriptions; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also provides support services and Microsoft consulting services 

In [2]:
import numpy as np
import pandas as pd

spark.conf.set("spark.sql.execution.arrow.enabled","true")


In [3]:
df = msft.history(period="5d")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-05-04,174.49,179.00,173.80,178.84,30372900,0,0
2020-05-05,180.62,183.65,179.90,180.76,36839200,0,0
2020-05-06,182.08,184.20,181.63,182.54,32139300,0,0
2020-05-07,184.17,184.55,182.58,183.60,28255500,0,0
2020-05-08,184.98,185.00,183.36,184.68,25605806,0,0


In [4]:
sparkdf = spark.createDataFrame(df)

/Users/psehgal/dev/spark/env/spark-2.4.5-bin-hadoop2.7/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [5]:
sparkdf

DataFrame[Open: double, High: double, Low: double, Close: double, Volume: bigint, Dividends: bigint, Stock Splits: bigint]

In [15]:
sparkdf.show()

+------+------+------+------+--------+---------+------------+
|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+------+------+------+------+--------+---------+------------+
|174.49| 179.0| 173.8|178.84|30372900|        0|           0|
|180.62|183.65| 179.9|180.76|36839200|        0|           0|
|182.08| 184.2|181.63|182.54|32139300|        0|           0|
|184.17|184.55|182.58| 183.6|28255500|        0|           0|
|184.98| 185.0|183.36|184.68|25605806|        0|           0|
+------+------+------+------+--------+---------+------------+



Define MySQL connection properties

In [9]:
sparkdf.write.jdbc(
    url='jdbc:mysql://localhost:3306/stocks',
    table='mystock',
    mode='append',
    properties={'user':'root','password':'yourpassword','driver':'com.mysql.cj.jdbc.Driver'}

)

Py4JJavaError: An error occurred while calling o60.jdbc.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$5.apply(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$5.apply(JDBCOptions.scala:99)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:190)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:194)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcR

In [12]:
sparkdf.write.format('jdbc').options(
        url='jdbc:mysql://localhost:3306/stocks',
        driver='com.mysql.jdbc.Driver',
        dbtable='mystock',
        user='root',
        password='yourpassword').mode('append')
#.mode('append').save()

In [13]:
private static final String MYSQL_USERNAME = "root";
private static final String MYSQL_PWD = "yourpassword";
private static final String MYSQL_CONNECTION_URL = "jdbc:mysql://localhost:3306/employees?user=" + MYSQL_USERNAME + "&password=" + MYSQL_PWD;

SyntaxError: invalid syntax (<ipython-input-13-657fa569af57>, line 1)

Create Spark context and sql context

In [14]:
private static final JavaSparkContext sc = new JavaSparkContext(new SparkConf().setAppName("SparkSaveToDb").setMaster("local[*]"));
private static final SQLContext sqlContext = new SQLContext(sc);

SyntaxError: invalid syntax (<ipython-input-14-20861046c35f>, line 1)